In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import autopep8

In [ ]:
autopep8 --in-place --aggressive --aggressive

In [ ]:
conda install -c conda-forge jupyter_nbextensions_configurator

## Проект 5

Необходимо собрать данные о продаже подержанных автомобилей, подготовить их, и на их основе обучить модель, которая будет предсказывать стоимость авто.

Начинаем с импорта необходимых библиотек:

In [ ]:
import numpy as np
import pandas as pd
import ast

from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR


from pprint import pprint
import pandas_profiling
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import tqdm

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

## Setup

In [ ]:
VERSION    = 11
DIR_TRAIN  = '../input/train27000/' # подключил к ноутбуку свой внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.33   # 33%
N_FOLDS    = 5

# CATBOOST
ITERATIONS = 2000
LR         = 0.1

## Данные

In [ ]:
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

Исследуем данные:

In [ ]:
test.info()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
test.sample(5)

Сразу можем отбросить колонки image, car_url и description - для построения модели они не подойдут

In [ ]:
test.drop(['image','car_url','description'], axis=1, inplace=True)

Для дальнейшего исследования используем мощный инструмент - pandas_profiling. Он поможет нам провести анализ датасета, определить типы переменных, распределения, количество пропусков внутри признака, количество уникальных значений и многое другое.

In [ ]:
#pandas_profiling.ProfileReport(test)

In [ ]:
test.columns

**bodyType**

Тип кузова. Категориальная переменная с 24 различными значениями

**brand**

Категориальная переменная, название производителя автомобиля. 12 различных значений

**color**

Цвет автомобиля. Категориальная переменная с 16 значениями.

**comlectation_dict**

Представляет собой словарь c характеристиками авто. К сожалению, высокое количество пропусков не позволит использовать данный признак. Убираем его: 

In [ ]:
test.drop('complectation_dict', axis=1, inplace=True)

**engineDisplacement**

Объем двигателя. Численная переменная, записанная в виде строки. Преобразуем значения для последующего исследования:

In [ ]:
test.engineDisplacement.value_counts()

In [ ]:
def strip_displace(string):
    num = 0
    try: num = float(string.strip(' LTR'))
    except: return 0
    
    return num

In [ ]:
test.engineDisplacement = test.engineDisplacement.apply(strip_displace)

**enginePower**

Мощность двигателя. Аналогично engineDisplacement:

In [ ]:
test.enginePower.value_counts()

In [ ]:
def strip_power(string):
    num = 0
    try: num = float(string.strip(' N12'))
    except: return 0
    
    return num

In [ ]:
test.enginePower = test.enginePower.apply(strip_power)

**equipment_dict**

Аналогично comlectation_dict содержит характеристики автомобиляб и также присутствует высокое количество пропущенных значений. Для простоты откажемся от этого признака:

In [ ]:
test.drop('equipment_dict', axis=1, inplace=True)

**fuelType**

Тип топлива. Категориальная переменная, 5 значений.

**mileage**

Пробег. Численная переменнаяю Распределение смещено в сторону нуля.

**modelDate**

Год производства модели. Численная переменная, которую удобнее использовать как категориальную. Присутствует несколько чрезвычайно низких значений, что может означать как ошибку при записи, так и наличие очень старых машин в списке

In [ ]:
test.modelDate.sort_values()

In [ ]:
test.iloc[16944]

В списке действительно присутствует чрезычайно старый автомобиль - Mercedes Simplex. Не имеется данных о его стоимости, но, как для раритетного, она наверняка не подчиняется стандартным правилам. Оценим количество старых автомобилей:

In [ ]:
len(test[test.modelDate < 1980])/len(test)

**model_info**

Словарь с информацией о данной модели. В отличие от предыдущихб не содержит пропусков. Рассмотрим подробнее:

In [ ]:
test.model_info[0]

Из ценного для нас - только само название модели, которое уже можно найти в следующей колонке. Отбрасываем признак: 

In [ ]:
test.drop('model_info', axis=1, inplace=True)

**model_name**

Название модели автомобиля. Категориальная переменная с 544 различными значениями. Рассмотрим:

In [ ]:
test.model_name.value_counts()

Посмотрим, какую долю из общего числа составляют 100 наиболее популярных моделей:

In [ ]:
#test.model_name.value_counts()[:100].sum()/test.model_name.value_counts().sum()

**name**

Текстовый признак. Содержит данные об объеме двигателя, его мощности и коробке переключения передач. Поскольку все эти признаки уже имеются в других колонках, этот столбец можно удалить:

In [ ]:
test.drop('name', axis=1, inplace=True)

**numberOfDoors**

Количество дверей. Численная переменная, которую удобнее использовать как категориальную.

**parsing_unixtime**

Когда был произведен парсинг. Для анализа признак не нужен:

In [ ]:
test.drop('parsing_unixtime', axis=1, inplace=True)

**productionDate**

Когда был произведен данный автомобиль. Численная переменная, которую можно использовать как категориальную.

**sell_id**

Идентификаионный номер. Для анализа не нужен:

In [ ]:
test.drop('sell_id', axis=1, inplace=True)

**super_gen**

Словарь с данными об автомобиле. Рассмотрим:



In [ ]:
test.super_gen[0]

In [ ]:
test.engineDisplacement[0]

In [ ]:
test.enginePower[0]

Содержит данные о sell_id (не нужен), объеме двигателя (более точное значение, можно заменить), типе топлива (уже есть), типе привода, КПП (значения на английском, можно заменить), мощности двигателя в лошадиных силах (более удобное значениеб можно заменить) и в киловаттах; некое "ускорение", значение клиренса и скорость расхода топлива в литрах на 100 км

In [ ]:
# Проверим, действительно ли даные представляют собой словарь
type(test.super_gen[0])

Воспользуемся библиотекой **ast** для преобразования строк в словари:

In [ ]:
test.super_gen = test.super_gen.apply(lambda x: ast.literal_eval(x))

Попробуем извлечь необходимые данные по ключам:

In [ ]:
test.engineDisplacement = test.super_gen.apply(lambda x: x['displacement'])

In [ ]:
test.fuelType = test.super_gen.apply(lambda x: x['engine_type'])

In [ ]:
test['Привод'] = test.super_gen.apply(lambda x: x['gear_type'])

In [ ]:
test.vehicleTransmission = test.super_gen.apply(lambda x: x['transmission'])

In [ ]:
test.enginePower = test.super_gen.apply(lambda x: x['power'])

В конце этот признак можно удалить:

In [ ]:
test.drop('super_gen', axis=1, inplace=True)

**vehicleConfiguration**

In [ ]:
test.vehicleConfiguration[0]

Строка, содержащая данные о типе кузоваб объеме двигателя и типе КПП, что мы уже имеем. Отбрасываем:

In [ ]:
test.drop('vehicleConfiguration', axis=1, inplace=True)

**vehicleTransmission**

Категориальная переменная, содержащая тип коробки передач (4 типа)

**vendor**

Категориальная переменная, продавец

**Владельцы**

Количество предыдущих владельцев. Категориальная переменная

**Владение**

Как долго машина у текущего владельца. Высокое количество пропусков, отбрасываем:

In [ ]:
test.drop('Владение', axis=1, inplace=True)

**ПТС**

Данные о ПТС. Категориальная переменная (2 значения). Присутствует 1 пропуск. Заполним:

In [ ]:
test['ПТС'] = test['ПТС'].fillna('Оригинал')

**Привод**

Тип привода, категориальный признак (3 значения).

**Руль**

Положение руля. Категориальный признак (2 значения).

**Состояние**

Требует ли автомобиль ремонта. Значение неизменно

**Таможня**

Растаможен ли автомобиль. Постоянное значение. Отбрасываем:

In [ ]:
test.drop('Таможня', axis=1, inplace=True)

**priceCurrency**

В признаке priceCurrency присутствует только одно значение - RUB. Очевидно, все автомобили, данные о которых собраны, продавались в рублях. Поскольку мы собираем данне о подержанных автомобилях, то логично предположить, что для подавляющего большинства валютой будет рубль. Этот признак удаляем:

In [ ]:
test.drop('priceCurrency', axis=1, inplace=True)

Взглянем на оставшиеся признаки:

In [ ]:
test.columns

In [ ]:
# Переименуем некоторые колонки для удобства:
test.columns=['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower',
       'fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors',
       'productionDate', 'transmission', 'vendor',
       'owners_number', 'PTS', 'wheel_drive', 'wheel', 'state']

In [ ]:
# Еще раз рассмотрим наш датасет

pandas_profiling.ProfileReport(test)

На основе данного датасета был произведен парсинг с сайта auto.ru. Данные из него мы используем для обучения:

In [ ]:
train = pd.read_csv(DIR_TRAIN+'final_train1.csv', sep=';', encoding='cp1251')

In [ ]:
# Код парсера

# Функция для подгонки hex-цвета под название

# import webcolors
# def closest_colour(requested_colour):
#     min_colours = {}
#     for key, name in webcolors.css3_hex_to_names.items():
#         r_c, g_c, b_c = webcolors.hex_to_rgb(key)
#         rd = (r_c - requested_colour[0]) ** 2
#         gd = (g_c - requested_colour[1]) ** 2
#         bd = (b_c - requested_colour[2]) ** 2
#         min_colours[(rd + gd + bd)] = name
#     return min_colours[min(min_colours.keys())]


# import requests
# import time
# import csv
# import re

# mark = input('Введите марку авто: ')
# FILE = 'cars-'+mark+'.csv'
# with open(FILE, 'a', newline='') as file:
#             writer = csv.writer(file, delimiter=';')
#             writer.writerow(['bodyType',
#                              'brand',
#                              'color',
#                              'engineDisplacement',
#                              'enginePower',
#                              'fuelType',
#                              'mileage',
#                              'modelDate',                            
#                              'model_name',
#                              'numberOfDoors',
#                              'productionDate',
#                              'transmission',
#                              'vendor',
#                              'owners_number',
#                              'PTS',
#                              'wheel_drive',
#                              'wheel',
#                              'state',
#                              'price'])
# a = 1 #Переменная для перехода по страницам
# cars=[]
# while a <= 99:
#     time.sleep(0.1)
#     #Объявление переменных как глобальные
#     global Name_auto, Marka_info, Color, Color_hex, Displacement, Power, Fuel
#     global Mileage, YearFrom, Model_info, Count_doors, Year
#     global Transmission, Vendor, Owners_number, PTS, Drive
#     global Wheel, State, Price_rub
    
#     URL = 'https://auto.ru/-/ajax/desktop/listing/' #URL на который будет отправлен запрос

#     #Параметры запроса
#     PARAMS = {
#         'catalog_filter' : [{"mark": mark.upper()}],
#          'section': "used",
#          'category': "cars",
#          'sort': "fresh_relevance_1-desc",
#          'page': a
#         }
#     #Заголовки страницы
#     HEADERS = {
#         'Accept': '*/*',
#         'Accept-Encoding': 'gzip, deflate, br',
#         'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
#         'Connection': 'keep-alive',
#         'Content-Length': '137',
#         'content-type': 'application/json',
#         'Cookie': '_csrf_token=1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24; autoru_sid=a%3Ag5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270%7C1580931467355.604800.8HnYnADZ6dSuzP1gctE0Fw.cd59AHgDSjoJxSYHCHfDUoj-f2orbR5pKj6U0ddu1G4; autoruuid=g5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270; suid=48a075680eac323f3f9ad5304157467a.bc50c5bde34519f174ccdba0bd791787; from_lifetime=1580933172327; from=yandex; X-Vertis-DC=myt; crookie=bp+bI7U7P7sm6q0mpUwAgWZrbzx3jePMKp8OPHqMwu9FdPseXCTs3bUqyAjp1fRRTDJ9Z5RZEdQLKToDLIpc7dWxb90=; cmtchd=MTU4MDkzMTQ3MjU0NQ==; yandexuid=1758388111580931457; bltsr=1; navigation_promo_seen-recalls=true',
#         'Host': 'auto.ru',
#         'origin': 'https://auto.ru',
#         'Referer': 'https://auto.ru/ryazan/cars/mercedes/all/',
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
#         'x-client-app-version': '202002.03.092255',
#         'x-client-date': '1580933207763',
#         'x-csrf-token': '1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24',
#         'x-page-request-id': '60142cd4f0c0edf51f96fd0134c6f02a',
#         'x-requested-with': 'fetch'
#     }

#     response = requests.post(URL, json=PARAMS, headers=HEADERS) #Делаем post запрос на url
#     try:
#         data = response.json()['offers'] #Переменная data хранит полученные объявления
#     except:
#         continue

#     #img_url = [] #Словарь в котором будут все картинки

#     i = 0 #Переменная для перехода по объявлениям
#     while i <= len(data) - 1: #len(data)-1 это количество пришедших объявлений


#         #Цвет автомобиля
# #         try: Color = str(data[i]['vehicle_info']['complectation']['vendor_colors'][0]['stock_color']['name_ru'])
# #         except: Color = 'Not color'
            
#         #Цвет автомобиля (возвращается в формате hex)
#         try: Color_hex = str(data[i]['color_hex'])
#         except: Color_hex = 'Not color'
        
#         try: Color = closest_colour(tuple(int(Color_hex[i:i+2], 16) for i in (0, 2, 4)))
#         except: Color = 'Unknown'
            
#         # Объем двигателя
#         try: Displacement = str(data[i]['vehicle_info']['tech_param']['displacement'])
#         except: Displacement = 'Unknown'
        
#         # Мощность двигателя
#         try: Power = str(data[i]['vehicle_info']['tech_param']['power'])
#         except: Power = 'Unknown'
        
#         # Тип топлива
#         try: Fuel = str(data[i]['vehicle_info']['tech_param']['engine_type'])
#         except: Fuel = 'Unknown'
            
#         # Коробка передач
#         try: Transmission = str(data[i]['vehicle_info']['tech_param']['transmission'])
#         except: Transmission = 'Unknown'
        
#         # Колличество владельцев автомобиля
#         try: Owners_number = str(data[i]['documents']['owners_number'])
#         except: Owners_number = 'The number of owners is not specified'
            
#         # Привод
#         try: Drive = str(data[i]['vehicle_info']['tech_param']['gear_type'])
#         except: Drive = 'Unknown'

#         # Дата выпуска модели
#         try: YearFrom = str(data[i]['vehicle_info']['super_gen']['year_from'])
#         except: YearFrom = 'Not year'
        
#         #PTS автомобиля
#         try: PTS = str(data[i]['documents']['pts']) #.encode(encoding='UTF-8')
#         except: PTS = 'Not PTS'


#         #Год выпуска автомобиля
#         try: Year = str(data[i]['documents']['year'])
#         except: Year = 'Not year'

#         #Цена в рублях
#         try: Price_rub = str(data[i]['price_info']['RUR'])
#         except: Price_rub = 'Not price rub'
            
#         # Продавец
#         try: Vendor = str(data[i]['vehicle_info']['vendor'])
#         except: Vendor = 'Unknown'
            
#         # Требуется ли ремонт
#         try: State = str(data[i]['state']['state_not_beaten'])
#         except: State = 'Unknown'
        
#         # Положение руля
#         try: Wheel = str(data[i]['vehicle_info']['steering_wheel'])
#         except: Wheel = 'Unknown'


#         #Пробег автомобиля
#         try: Mileage = str(data[i]['state']['mileage']) #.encode(encoding='UTF-8')
#         except: Mileage = 'Not mileage'


#         #Количество дверей у автомобиля
#         try: Count_doors = str(data[i]['vehicle_info']['configuration']['doors_count'])#.encode(encoding='UTF-8')
#         except: Count_doors = 'Not count doors'

#         #Название автомобиля
#         try: Name_auto = str(data[i]['vehicle_info']['configuration']['human_name'])
#         except: Name_auto = 'Not name auto'


#         #Марка автомобиля
#         try: Marka_info = str(data[i]['vehicle_info']['mark_info']['code']) #.encode(encoding='UTF-8')
#         except: Marka_info = 'Not marka info'

#         #Модель автомобиля
#         try: Model_info = str(data[i]['vehicle_info']['model_info']['code']) #.encode(encoding='UTF-8')
#         except: Model_info = 'Not model info'
        
        
#         #Записываем переменную в файл
#         with open(FILE, 'a', newline='') as file:
#             writer = csv.writer(file, delimiter=';')
#             writer.writerow([Name_auto,
#                              Marka_info,
#                              Color,
#                              re.sub(r'[^\x00-\x7f]', '', Displacement),
#                              re.sub(r'[^\x00-\x7f]', '', Power),
#                              re.sub(r'[^\x00-\x7f]', '', Fuel),
#                              re.sub(r'[^\x00-\x7f]', '', Mileage),
#                              re.sub(r'[^\x00-\x7f]', '', YearFrom),                            
#                              Model_info,
#                              re.sub(r'[^\x00-\x7f]', '', Count_doors),
#                              re.sub(r'[^\x00-\x7f]', '', Year),
#                              re.sub(r'[^\x00-\x7f]', '', Transmission),
#                              re.sub(r'[^\x00-\x7f]', '', Vendor),
#                              re.sub(r'[^\x00-\x7f]', '', Owners_number),
#                              re.sub(r'[^\x00-\x7f]', '', PTS),
#                              re.sub(r'[^\x00-\x7f]', '', Drive),
#                              re.sub(r'[^\x00-\x7f]', '', Wheel),
#                              re.sub(r'[^\x00-\x7f]', '', State),
#                              re.sub(r'[^\x00-\x7f]', '', Price_rub)])        
        
        
#         i += 1 #Увеличиваем переменную перехода по объявлениям на 1
#     print('Page: ' + str(a)) #Выводим сообщение, какая страница записалась
#     a += 1 #Увеличиваем переменную страницы сайта на 1

# print('Successfully') #Выводим информацию об успешном выполнении}

In [ ]:
test.drop('state', axis=1, inplace=True)

In [ ]:
train.drop(['df-index', 'state'], axis=1, inplace=True)

In [ ]:
train.sample(5)

Напишем функции для обработки train и test и их приведения к общему виду:

In [ ]:
# Преобразуем строки в числа, где возможно. Если не возможно - отмечаем особым значением
def to_int(string):
    try: return(int(string))
    except: return 1234567
    
# Преобразование PTS
def pts(string):
    if string == 'Оригинал':
        string = 'ORIGINAL'
    elif string == 'Дубликат':
        string = 'DUPLICATE'
    else:
        string = string
        
    return string

# Преобразование wheel
def wheel(string):
    if string == 'Левый':
        string = 'LEFT'
    elif string == 'Правый':
        string = 'RIGHT'
    else:
        string = string
        
    return string


def preprocessing(data, flag=0):
    # Для bodyType понизим регистр и оставим только первую часть;
    data.bodyType = data.bodyType.apply(lambda x: x.lower())
    data.bodyType = data.bodyType.apply(lambda x: x.split()[0])

    # Для model_name повысим регистр;
    data.model_name = data.model_name.apply(lambda x: x.upper())

    # Получилось спарсить только цвета на английском, поэтому уберем color;
    data.drop('color', axis=1, inplace=True)

    # При парсинге многие данные были получены в строковой форме.
    # Переведем в числа, где необходимо;
    data.mileage = data.mileage.apply(to_int)
    data.engineDisplacement = data.engineDisplacement.apply(to_int)
    data.enginePower = data.enginePower.apply(to_int)
    data.modelDate = data.modelDate.apply(to_int)
    data.productionDate = data.productionDate.apply(to_int)
    data.numberOfDoors = data.numberOfDoors.apply(to_int)
    
    # Для test не нужна обработка price
    if flag == 1:
        data.price = data.price.apply(to_int)    

    # Преобразуем значения wheel и PTS к единой форме
    data.PTS = data.PTS.apply(pts)
    data.wheel = data.wheel.apply(wheel)
    
    # Убираем лишнюю запись, содержащуюю в качестве значений заголовок 
    # датафрейма;
    data = data[data.owners_number != 'owners_number']
    
    # Преобразуем значения owners_number к единой форме;
    data.owners_number = data.owners_number.apply(lambda x: int(x.strip()[0]))
    
    return data

In [ ]:
train = preprocessing(train, 1)

In [ ]:
train = train[train.price != 1234567]

In [ ]:
test = preprocessing(test, 0)

In [ ]:
train.sample(5)

In [ ]:
test.sample(5)

## Общее исследование

In [ ]:
# объединим train и test
test['price'] = -100
data = test.append(train, ignore_index=True)

In [ ]:
pandas_profiling.ProfileReport(data)

Высокая корреляция между modelDate и productionDate. Оценим:

In [ ]:
data.modelDate.corr(data.productionDate)

In [ ]:
# Корреляция более 90%, но отбрасывать не будем - ухудшает результат предсказаний

In [ ]:
# Оценим важность непрерывных переменных
num_cols=['engineDisplacement','enginePower', 'mileage', 'modelDate', 'productionDate']

imp_num = pd.Series(f_classif(train[num_cols], train.price)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [ ]:
# Преобразуем категориальные признаки в тип данных cat и оценим их важность
cat_cols = ['bodyType', 'brand', 'fuelType',
        'model_name', 'numberOfDoors', 'transmission',
       'vendor', 'owners_number', 'PTS', 'wheel_drive', 'wheel']

for column in cat_cols:
    train[column] = train[column].astype('category').cat.codes
    data[column] = data[column].astype('category').cat.codes

imp_cat = pd.Series(mutual_info_classif(train[cat_cols], train.price,
                                     discrete_features =True), index = cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [ ]:
# Наименьшую важность имеют признаки PTS и wheel

## Train Split

Учитывая форму распределения price, можно попробовать также логарифмировать этот показатель. Попробуем обучение и так, и так.

In [ ]:
# Разъединим train и test

train = data[data.price != -100]
test = data[data.price == -100]

In [ ]:
X_sub = test.drop('price', axis=1)

In [ ]:
X = train.drop('price', axis=1)
y = train.price.values
y_log = train.price.apply(np.log).values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X, y_log, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
y_train_log

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [ ]:
1/0

## Наивная модель

Наивная модель для предсказания будет использовать только колонку **enginePower**

In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
mape(y_test, X_test['enginePower'].map(tmp_train.groupby('enginePower')['price'].median()).values)

In [ ]:
predict = X_test['enginePower'].map(tmp_train.groupby('enginePower')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(np.mean(np.abs((y_test-predict)/y_test)))*100:0.2f}%")

## RandomForest

In [ ]:
random_forest = RandomForestRegressor(random_state = RANDOM_SEED)

In [ ]:
rf = random_forest.fit(X_train, y_train)
rf_log = random_forest.fit(X_train_log, y_train_log)

In [ ]:
rf_sub = rf.predict(X_test)
rf_log_sub = rf_log.predict(X_test_log)

In [ ]:
mape(rf_sub, y_test)

При подсчете MAPE, похоже, образовалась какая-то проблема. Используем прямую формулу:

In [ ]:
np.mean(np.abs((y_test-rf_sub)/y_test))

In [ ]:
mape(rf_log_sub, y_test_log)

In [ ]:
np.mean(np.abs((y_test_log-rf_log_sub)/y_test_log))

Логарифмированные данные показывают высокие значения. Попробуем кросс-валидацию

**Кросс-валидация**

In [ ]:
def forest_model(y_train, X_train, X_test, y_test):
    model = RandomForestRegressor(random_state = RANDOM_SEED)
    model.fit(X_train, y_train)
    
    return(model)

In [ ]:
submissions = pd.DataFrame(0,columns=["sub_1"], index=sample_submission.index) # куда пишем предикты по каждой модели
score_ls = []
splits = list(KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X, y))

for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=N_FOLDS,):
    # use the indexes to extract the folds in the train and validation data
    X_train, y_train, X_test, y_test = X.iloc[train_idx], y[train_idx], X.iloc[test_idx], y[test_idx]
    # model for this fold
    model = forest_model(y_train_log, X_train_log, X_test_log, y_test_log)
    # score model on test
    test_predict = model.predict(X_test_log)
    test_score = mape(y_test_log, test_predict)
    score_ls.append(test_score)
    print(f"{idx+1} Fold Test MAPE: {mape(y_test_log, test_predict):0.3f}")
    # submissions
    submissions[f'sub_{idx+1}'] = model.predict(X_sub)
    #model.save_model(f'catboost_fold_{idx+1}.model')
    
print(f'Mean Score: {np.mean(score_ls):0.3f}')
print(f'Std Score: {np.std(score_ls):0.4f}')
print(f'Max Score: {np.max(score_ls):0.3f}')
print(f'Min Score: {np.min(score_ls):0.3f}')

In [ ]:
submissions['blend'] = (submissions.sum(axis=1))/len(submissions.columns)
sample_submission['price'] = np.exp(submissions['blend'].values)
sample_submission.to_csv(f'forest_blend_v12.csv', index=False)
sample_submission.head(10)

Фактический MAPE - 22.48%

## CatBoost. Сat features

In [ ]:
X_train.nunique()

In [ ]:
cat_features_ids = np.where(X_train.apply(pd.Series.nunique) < 3000)[0].tolist()

In [ ]:
cat_features_ids

## Fit

In [ ]:
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAPE']
                         )
model.fit(X_train, y_train,
         cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=200,
         use_best_model=True,
         plot=True
         )

In [ ]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

In [ ]:
model_log = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAPE']
                         )
model_log.fit(X_train_log, y_train_log,
         cat_features=cat_features_ids,
         eval_set=(X_test_log, y_test_log),
         verbose_eval=200,
         use_best_model=True,
         plot=True
         )

In [ ]:
# оцениваем точность
predict_log = model_log.predict(X_test_log)
print(f"Точность модели по метрике MAPE: {(mape(y_test_log, predict_log))*100:0.2f}%")

Логарифмирование показало значительное улучшение точности предсказаний CatBoost. Проверим, действительно ли точность повышается:

In [ ]:
predict_submission = model_log.predict(X_sub)
predict_submission

In [ ]:
sample_submission['price'] = np.exp(predict_submission)
sample_submission.to_csv(f'log_v1.csv', index=False)
sample_submission.head(10)

Результат - 23% MAPE. Улучшение, но не такое значительное, как можно было надеяться. Попробуем кросс-валидацию.

**Кросс-валидация**

In [ ]:
def cat_model(y_train, X_train, X_test, y_test):
    model = CatBoostRegressor(iterations = ITERATIONS,
                              learning_rate = LR,
                              eval_metric='MAPE',
                              random_seed = RANDOM_SEED,)
    model.fit(X_train, y_train,
              cat_features=cat_features_ids,
              eval_set=(X_test, y_test),
              verbose=False,
              use_best_model=True,
              plot=False)
    return(model)

In [ ]:
submissions = pd.DataFrame(0,columns=["sub_1"], index=sample_submission.index) # куда пишем предикты по каждой модели
score_ls = []
splits = list(KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X, y))

for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=N_FOLDS,):
    # use the indexes to extract the folds in the train and validation data
    X_train, y_train, X_test, y_test = X.iloc[train_idx], y[train_idx], X.iloc[test_idx], y[test_idx]
    # model for this fold
    model = cat_model(y_train_log, X_train_log, X_test_log, y_test_log,)
    # score model on test
    test_predict = model.predict(X_test_log)
    test_score = mape(y_test_log, test_predict)
    score_ls.append(test_score)
    print(f"{idx+1} Fold Test MAPE: {mape(y_test_log, test_predict):0.3f}")
    # submissions
    submissions[f'sub_{idx+1}'] = model.predict(X_sub)
    model.save_model(f'catboost_fold_{idx+1}.model')
    
print(f'Mean Score: {np.mean(score_ls):0.3f}')
print(f'Std Score: {np.std(score_ls):0.4f}')
print(f'Max Score: {np.max(score_ls):0.3f}')
print(f'Min Score: {np.min(score_ls):0.3f}')

**Submissions blend**

In [ ]:
submissions.head(10)

In [ ]:
submissions['blend'] = (submissions.sum(axis=1))/len(submissions.columns)
sample_submission['price'] = np.exp(submissions['blend'].values)
sample_submission.to_csv(f'submission_blend_v15.csv', index=False)
sample_submission.head(10)

## Стекинг

In [ ]:
reg1 =  CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAPE'],
                          cat_features=cat_features_ids,
                          #eval_set=(X_test, y_test),
                          verbose=False,
                          #use_best_model=True,
                          #plot=False
                         )
reg2 = RandomForestRegressor(random_state=RANDOM_SEED)
reg3 = Ridge(random_state=RANDOM_SEED)


lr = LinearRegression()

sreg = StackingRegressor(regressors=[reg1, reg2, reg3],
                         meta_regressor=lr)

In [ ]:
sreg.fit(X_train_log, y_train_log)

In [ ]:
mape(sreg.predict(X_test_log), y_test_log)

## Submission

In [ ]:
predict_submission = sreg.predict(X_sub)
predict_submission

In [ ]:
sample_submission['price'] = np.exp(predict_submission)
sample_submission.to_csv(f'stacking_v2.csv', index=False)
sample_submission.head(10)

Фактический MAPE - 24.9%